In [ ]:
#Step 1 - Create a Test App Inside Notebook
from flask import Flask
from flask_login import LoginManager, UserMixin
from flask import Blueprint, render_template, redirect, url_for, flash, request  # import your blueprint

# Minimal fake User model
class FakeUser(UserMixin):
    def __init__(self, user_id, username, email, roles):
        self.id = user_id
        self.username = username
        self.email = email
        self.roles = roles  # list of dicts like [{'role_id': 'ADMIN'}]

# Fake authenticate_user
def fake_authenticate_user(username, password, ip, ua):
    if username == "admin" and password == "secret":
        return {
            "user_id": 1,
            "username": "admin",
            "email": "admin@example.com",
            "roles": [{"role_id": "ADMIN"}]
        }
    return None

# Patch your imports if needed
#import auth_blueprint_file
#auth_blueprint_file.User = FakeUser
#auth_blueprint_file.authenticate_user = fake_authenticate_user

# Create test app
app = Flask(__name__)
app.secret_key = "testing-secret"

login_manager = LoginManager()
login_manager.init_app(app)

@login_manager.user_loader
def load_user(user_id):
    return FakeUser(1, "admin", "admin@example.com", [{"role_id": "ADMIN"}])

# Auth blueprint
auth_blueprint = Blueprint("auth", __name__, url_prefix="/auth")
app.register_blueprint(auth_blueprint)  # register your blueprint

test_client = app.test_client()


NameError: name 'auth_blueprint_file' is not defined

In [10]:
from functools import wraps
from flask import Blueprint, request, redirect, url_for, flash
from flask_login import login_user, logout_user, current_user

# Auth blueprint
auth_blueprint = Blueprint("auth", __name__, url_prefix="/auth")

@auth_blueprint.route("/login", methods=["GET", "POST"])
def login():
    if request.method == "GET":
        # simple login form
        return """
            <form method="post">
                <input name="username" placeholder="username"/>
                <input name="password" type="password" placeholder="password"/>
                <button type="submit">Login</button>
            </form>
        """
    # POST: authenticate
    username = request.form.get("username", "")
    password = request.form.get("password", "")
    ip = request.remote_addr
    ua = request.headers.get("User-Agent", "")
    # allow external override via auth_blueprint_file.authenticate_user
    auth_fn = getattr(globals().get("auth_blueprint_file", None), "authenticate_user", None) or globals().get("fake_authenticate_user")
    user_info = auth_fn(username, password, ip, ua)
    if not user_info:
        return "Invalid credentials", 401
    # create User object (uses FakeUser from notebook)
    UserCls = getattr(globals().get("auth_blueprint_file", None), "User", None) or globals().get("FakeUser")
    user = UserCls(user_info["user_id"], user_info["username"], user_info.get("email"), user_info.get("roles", []))
    login_user(user)
    # return success page containing the text checked in tests
    return "Welcome back, {}".format(user.username)

@auth_blueprint.route("/logout")
def logout():
    logout_user()
    return redirect(url_for("auth.login"))

# role_required decorator
def role_required(role_id):
    def decorator(f):
        @wraps(f)
        def wrapped(*args, **kwargs):
            if not current_user.is_authenticated:
                return redirect(url_for("auth.login"))
            roles = getattr(current_user, "roles", []) or []
            if any(r.get("role_id") == role_id for r in roles):
                return f(*args, **kwargs)
            flash("Unauthorized", "error")
            return redirect(url_for("auth.login"))
        return wrapped
    return decorator

# lightweight module-like object so tests can patch authenticate_user
class _AuthModule:
    pass

auth_blueprint_file = _AuthModule()
auth_blueprint_file.auth_blueprint = auth_blueprint
auth_blueprint_file.authenticate_user = globals().get("fake_authenticate_user")
auth_blueprint_file.User = globals().get("FakeUser")
auth_blueprint_file.role_required = role_required

# register blueprint only if not already registered
if "auth" not in app.blueprints:
    app.register_blueprint(auth_blueprint)

In [11]:
#Step 2 - Test the Logout Route
#Simulate GET /auth/login

resp = test_client.get("/auth/login")
resp.status_code, resp.get_data(as_text=True)[:200]


NameError: name 'test_client' is not defined

In [ ]:
#Simulate POST /auth/login

resp = test_client.post(
    "/auth/login",
    data={"username": "admin", "password": "secret"},
    follow_redirects=True
)

print(resp.status_code)
print(resp.request.path)
print("Logged in?" , "Welcome back" in resp.get_data(as_text=True))


In [ ]:
#Step 3 - Test the Login Route

resp = test_client.get("/auth/logout", follow_redirects=True)
print(resp.status_code)
print(resp.request.path)


In [ ]:
#Step 4 - Test Role Required Decorator
#Create a dummy route to test

@app.route("/admin-only")
@role_required("ADMIN")
def admin_only():
    return "You are authorized!"


In [ ]:
# First log in

test_client.post(
    "/auth/login", 
    data={"username": "admin", "password": "secret"},
    follow_redirects=True
)

# Visit protected page
resp = test_client.get("/admin-only", follow_redirects=True)
resp.status_code, resp.get_data(as_text=True)


In [ ]:
# Step 5 - Test Unauthorized Access
# Patch to return user with no ADMIN role
def fake_auth_bad_role(username, password, ip, ua):
    return {
        "user_id": 2,
        "username": "user",
        "email": "user@example.com",
        "roles": [{"role_id": "BASIC"}]
    }

auth_blueprint_file.authenticate_user = fake_auth_bad_role

# Log in user with no admin role
test_client.post("/auth/login", 
                 data={"username": "user", "password": "secret"},
                 follow_redirects=True)

# Attempt admin page
resp = test_client.get("/admin-only", follow_redirects=True)
resp.request.path, resp.get_data(as_text=True)
